In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# check to see if GPU
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import numpy as np
import math, random

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

import os, json, pickle

import torch
import detectron2

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.utils.logger import setup_logger
setup_logger()

!nvcc --version
TORCH_VERSION = (torch.__version__)
print("torch: ", TORCH_VERSION)
print("detectron2:", detectron2.__version__)

/bin/bash: line 1: nvidia-smi: command not found
Cloning into 'detectron2'...
remote: Enumerating objects: 15912, done.
remote: Total 15912 (delta 0), reused 0 (delta 0), pack-reused 15912 (from 1)
Receiving objects: 100% (15912/15912), 6.67 MiB | 16.69 MiB/s, done.
Resolving deltas: 100% (11332/11332), done.


W1003 14:04:38.186000 1210 torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=c5fa4939bb814416ce3c8fce97e37aee1e3bd7d872bcfdaf30a8bba9b68173a0
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
Successfully built fvcore
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.8.0+cu126
detectron2: 0.6


In [3]:
#@title load images
from detectron2.data.datasets import register_coco_instances
# base_path = "/content/drive/MyDrive/DLforPVFacades/paper/dataset/4_dataset_final_PVandNonPV/PVFinderPaper.v3-pvandnonpv.coco-segmentation/"
base_path = "/content/drive/MyDrive/DLforPVFacades/paper/dataset/5_dataset_final_PVandNonPV_ColorAug/PVFinderPaper.v7-pvandnonpv_augflipcropcolor.coco-segmentation"

for split in ["test"]:
    try:
        MetadataCatalog.remove(split)
    except:
        pass
    try:
        DatasetCatalog.remove(split)
    except:
        pass

    register_coco_instances(split, {}, os.path.join(base_path, split, "_annotations_case2.coco.json"), os.path.join(base_path, split))
    print(f"{split} with {len([item for item in os.listdir(os.path.join(base_path, split)) if item.endswith('.jpg')])}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DLforPVFacades/paper/dataset/5_dataset_final_PVandNonPV_ColorAug/PVFinderPaper.v7-pvandnonpv_augflipcropcolor.coco-segmentation/test'

In [ ]:
import pandas as pd
import requests
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from io import BytesIO
import json
import os
import re
import pprint
from detectron2.utils.visualizer import ColorMode
from detectron2 import structures
from pycocotools import mask as mask_utils
import csv
import gc
from tqdm.notebook import tqdm

In [ ]:
pp = pprint.PrettyPrinter()

csv_file = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/Batch_5301916_batch_results_fin.csv'
df = pd.read_csv(csv_file)

output_folder = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/output'
os.makedirs(output_folder, exist_ok=True)

masks_folder = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/masks'
os.makedirs(masks_folder, exist_ok=True)

dataset = 'test'
dataset_dicts = DatasetCatalog.get(dataset)
metadata = MetadataCatalog.get(dataset)

gt_dict = {}
for item in dataset_dicts:
    img_name = os.path.basename(item["file_name"]).split(".rf")[0]
    gt_dict[img_name] = item

results = []
for image_url, group in df.groupby('Input.image_url'):
    print(f"\n\n\n>>> img: {image_url}")


    '''
    LOAD GROUND TRUTH IMAGE
    '''
    img_name = image_url.split('.rf')[0]
    gt = gt_dict[img_name]

    print(f"h: {gt['height']}\tw:{gt['width']}")
    print(len(gt['annotations']))

    img = cv2.imread(gt["file_name"])

    # CREATE UNIFIED GT MASK
    gt_mask = None
    gt_union_mask = np.zeros_like(img[:, :, 0], dtype=int)
    for item in gt['annotations']:
        gt_mask = mask_utils.decode(item['segmentation']).astype(int)
        gt_union_mask = np.logical_or(gt_union_mask, gt_mask).astype(int)
    gt_mask = gt_union_mask

    '''
    BRING EACH WORKER'S ANNOTATION MASK (ANNOT)
    '''
    annotations = {}

    # for each worker, extract annotations
    for idx, row in group.iterrows():
        if row['AssignmentStatus'] != 'Approved':
            print(f"\n\n\nSKIPPING {row}")
            continue
        worker_id = row['WorkerId']
        polygons_data = row['Answer.annotatedResult.polygons']
        if pd.isna(polygons_data) or polygons_data == '[]':  # No annotation case
            annotations[worker_id] = None
        else:
            annotations[worker_id] = json.loads(polygons_data)



    for worker, polygons in annotations.items():
        polygons_vertices = []

        print(worker, polygons)
        if polygons is not None:
            for polygon in polygons:
                vertices = [(vertex['x'], vertex['y']) for vertex in polygon['vertices']]
                polygons_vertices.append(vertices)  # <-- This was wrongly indented

        pred_mask = np.zeros_like(img[:, :, 0], dtype=np.uint8)  # should be uint8 for cv2
        for polygon_vertices in polygons_vertices:
            pts = np.array([polygon_vertices], dtype=np.int32)  # must be wrapped in list
            cv2.fillPoly(pred_mask, pts, color=1)

        # Define the save directory
        img_folder_name = os.path.splitext(os.path.basename(image_url))[0]  # Remove .jpg
        save_dir = os.path.join(masks_folder, img_folder_name)

        # Create the directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Define the path for this worker's mask
        mask_filename = f"{worker}.png"  # or .jpg if you prefer
        mask_path = os.path.join(save_dir, mask_filename)

        print()

        # Save the mask
        cv2.imwrite(mask_path, (pred_mask * 255).astype(np.uint8))  # Save mask as binary image (0 or 255)

        # Convert BGR to RGB for Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()

        # Create base transparent background by dimming the image
        transparent_bg = (img_rgb * 0.2).astype(np.uint8)  # 20% brightness

        # Create overlay to add masks
        overlay = transparent_bg.copy()

        # Define colors
        gt_color = (255, 0, 0)          # Red
        pred_color = (255, 165, 0)      # Orange
        overlap_color = (0, 255, 0)     # Green

        # Overlay masks
        overlay[gt_mask == 1] = gt_color
        overlay[pred_mask == 1] = pred_color
        overlay[(gt_mask == 1) & (pred_mask == 1)] = overlap_color

        # Blend with the transparent base (which is already dimmed, so blending is optional)
        blended = overlay

        # Show both images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(img_rgb)
        axes[0].set_title("Original Image")
        axes[0].axis('off')

        axes[1].imshow(blended)
        axes[1].set_title(f"Worker {worker} vs GT")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        # DETECTION-LEVEL METRICS
        gt_detected = gt_mask.sum() > 0
        pred_detected = pred_mask.sum() > 0

        if gt_detected and pred_detected:
            det_category = 'TP'
        elif not gt_detected and pred_detected:
            det_category = 'FP'
        elif gt_detected and not pred_detected:
            det_category = 'FN'
        else:
            det_category = 'TN'

        # Calculate PIXEL-LEVEL TP, FP, FN, TN
        tp = np.logical_and(gt_mask == 1, pred_mask == 1).sum()
        fp = np.logical_and(gt_mask == 0, pred_mask == 1).sum()
        fn = np.logical_and(gt_mask == 1, pred_mask == 0).sum()
        tn = np.logical_and(gt_mask == 0, pred_mask == 0).sum()

        # Optionally calculate metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        iou = tp / (tp + fp + fn)

        print(f"\nMetrics for Worker {worker}:")
        print(f"TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}")
        print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}, IoU: {iou:.3f}")

        # Inside loop per image & worker:
        results.append({
            'image': image_url,
            'worker': worker,
            'det_category': det_category,
            'img_precision': precision,
            'img_recall': recall,
            'img_f1': f1,
            'img_iou': iou,
            'TP_pixels': tp,
            'FP_pixels': fp,
            'FN_pixels': fn,
            'TN_pixels': tn,
            })

# After the full loop:
metrics_df = pd.DataFrame(results)
metrics_df.to_csv(os.path.join(output_dir, 'humanVSai.csv'), index=False)

Output hidden; open in https://colab.research.google.com to view.